<a href="https://colab.research.google.com/github/Gauravhulmukh/Chatbot-Development-for-Regional-language-using-Artificial-Intelligence/blob/master/chatbotmodelcomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')#findout why we need punkt for word tokenziation? 
#Stemmer=just simple the word:helper=help
#it has two type porter and Lancaster


# things we need for Tensorflow
import numpy as np
!pip install tflearn
import tflearn
import tensorflow as tf
import random

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# import our chat-bot intents file
import json
with open('/content/drive/My Drive/intents.json') as json_data:
    intents = json.load(json_data)

In [0]:
#simply creating array
words = []
classes = []
documents = []
ignore_words = ['?']#it will ignore  question MARK from pattern

#create loop intents>patterns 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern) #seperate words:hi there=hi,there
        words.extend(w)
        #print(words) #it will add seperated words from each pattern into words array 
        documents.append((w, intent['tag']))
        #print(documents) #it is set of words(tokenize_pattern) & tag
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))
#create classes array which contain all the tags in alphabhetical order

print (len(documents), documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)



In [0]:
#creating arrays for training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    
    for w in words:
      #print(w,pattern_words)
        bag.append(1) if w in pattern_words else bag.append(0)
        #print(len(bag),bag) #it is comparing alphabhetical words array & non-alphabhetical stem words of document
  #match 1 else 0
        
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    #print(output_row)
    output_row[classes.index(doc[1])] = 1
    #print(len(output_row),output_row) #it is comparing classes array & doc[1] array

    #adding bag with 1&0 and
    training.append([bag, output_row])
    #print(len(bag),len(output_row,bag, output_row))
    #print(training)

    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)#findout meaning of it--------------------------
#print(training) #it is printing 1 on 39th and 78 for matching hi,ther first pattern of pattern dataset 


# create train and test lists
train_x = list(training[:,0])#it contain bag array of 0&1 #contain 88 unique stemmed words
train_y = list(training[:,1])#it contain  classes array of 9 in 0&1 #contain 9 classes

#print(len(train_x),train_x)
#print(train_y)
# refer this https://sourcedexter.com/tensorflow-text-classification-python/
#print(training)


In [0]:
#Clears the default graph stack and resets the global default graph.
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
#print(net) Tensor("InputData/X:0", shape=(?, 88), dtype=float32)

net = tflearn.fully_connected(net, 8)
#print(net) Tensor("FullyConnected/BiasAdd:0", shape=(?, 8), dtype=float32)

net = tflearn.fully_connected(net, 8)
#print(net)  Tensor("FullyConnected_1/BiasAdd:0", shape=(?, 8), dtype=float32)

net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
#print(net)  Tensor("FullyConnected_2/Softmax:0", shape=(?, 9), dtype=float32)

net = tflearn.regression(net)
#print(net)  #Tensor("FullyConnected_2/Softmax:0", shape=(?, 9), dtype=float32)


# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_chatbot_redsamurai_medical_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('chatbot_redsamurai_medical_model.tflearn')

#print(model)
#model.load('chatbot_invoicing_model.tflearn')


Training Step: 5999  | total loss: 0.16458 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.16458 - acc: 0.9921 -- iter: 40/45
Training Step: 6000  | total loss: 0.15619 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.15619 - acc: 0.9929 -- iter: 45/45
--
INFO:tensorflow:/content/chatbot_redsamurai_medical_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
#model.load('chatbot_invoicing_model.tflearn')
#print(model_path)

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
p = bow("I'm looking for pharmacy", words)
print (len(p),p)
#p = bow("thanks", words)
#q = bow("'s hi gaurav", words)
#print(words)
#print (len(q),q)

print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [0]:
print(model.predict([p]))

[[1.3359516e-02 7.2231139e-03 1.3467361e-01 1.4167568e-02 9.1557791e-03
  2.9979897e-01 1.0897052e-05 5.2159601e-01 1.4485136e-05]]


In [0]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "chatbot_redsamurai_medical_training_data", "wb" ) )